In [15]:
from pynq import Overlay
import asyncio
import time
import threading
import math
from threading import Thread
import inspect
import ctypes
import socket
global msg
global motor_rstflag
global motor_scanflag
global xpos_last,ypos_last
global scan_cnt,dir_scan
overlay=Overlay("./motor.bit")

ip ='192.168.2.1'
port = 777
motor_rstflag=0
motor_scanflag=0
xpos_last=0
ypos_last=0
scan_cnt=0
dir_scan=0

In [16]:
def _async_raise(tid, exctype):

    """raises the exception, performs cleanup if needed"""

    tid = ctypes.c_long(tid)

    if not inspect.isclass(exctype):

        exctype = type(exctype)

    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))

    if res == 0:

        raise ValueError("invalid thread id")

    elif res != 1:

        # """if it returns a number greater than one, you're in trouble,

        # and you should call it again with exc=NULL to revert the effect"""

        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)

        raise SystemError("PyThreadState_SetAsyncExc failed")

def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

In [17]:
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
def start_tcp_client(ip,port):
    failed_count = 0
    while True:
        try:
            print("start connect to server ")
            client.connect((ip,port))
            print("connect success ")
            break
        except socket.error:
            failed_count += 1
            print("fail to connect to server %d times" % failed_count)
            if failed_count == 100: return

start_tcp_client('192.168.2.1',777)

start connect to server 
connect success 


In [18]:
pwm0=overlay.pwmip_0
pwm1=overlay.pwmip_1
ioout=overlay.axi_gpio_0
ioin=overlay.axi_gpio_1

In [19]:
class motor_0:    
    def clear(self,x):
        pwm0.write(0x00,x)
        
    def cnt(self):
        print("当前计数值为"+str(pwmip.read(0x04)))
        
    def dire(self,x):
        if x:
            ioout.write(0x00,ioout.read(0x00)|1)
        else:
            ioout.write(0x00,ioout.read(0x00)&~1)
            
    def speed(self,x):
        pwm0.write(0x00,1)
        pwm0.write(0x08,x)
        pwm0.write(0x00,0)

    def step(self):
        return pwm0.read(0x04)
    
    def power_en(self,x):
        if x:
            ioout.write(0x00,ioout.read(0x00)|(1<<1))
        else:
            ioout.write(0x00,ioout.read(0x00)&~(1<<1)) 
        
    def pwm_en(self,x):
        pwm0.write(0x0c,x) 

    def stop(self):
        self.pwm_en(0)
        self.power_en(1)
        self.clear(1)
        
    def motor_run(self,d,v,s):
        self.power_en(0)
        self.clear(1)
        self.clear(0)
        self.dire(d)
        self.speed(v) 
        self.pwm_en(1)
        if s==0:
            pass
        else:
            while True:
                if (s*1280)<=self.step():
                    self.stop()
                    break
                    
    def location_rst(self):
        self.motor_run(0,5000,0)

In [20]:
class motor_1:    
    def clear(self,x):
        pwm1.write(0x00,x)
        
    def cnt(self):
        print("当前计数值为"+str(pwmip.read(0x04)))
        
    def dire(self,x):
        if x:
            ioout.write(0x00,ioout.read(0x00)|(1<<3))
        else:
            ioout.write(0x00,ioout.read(0x00)&~(1<<3))
            
    def speed(self,x):
        pwm1.write(0x00,1)
        pwm1.write(0x08,x)
        pwm1.write(0x00,0)
        
    def step(self):
        return pwm1.read(0x04)
    
    def power_en(self,x):
        if x:
            ioout.write(0x00,ioout.read(0x00)|(1<<2))
        else:
            ioout.write(0x00,ioout.read(0x00)&~(1<<2)) 
        
    def pwm_en(self,x):
        pwm1.write(0x0c,x) 
     
    def stop(self):
        self.pwm_en(0)
        self.power_en(1)
        self.clear(1)
        
    def motor_run(self,d,v,s):
        self.power_en(0)
        self.clear(1)
        self.clear(0)
        self.dire(d)
        self.speed(v) 
        self.pwm_en(1) 
        if s==0:
            pass
        else:
            while True:
                if (s*1280)<=self.step():
                    self.stop()
                    break
        
    def location_rst(self):
        self.motor_run(0,5000,0)

In [21]:
motor0=motor_0()
motor1=motor_1()

In [22]:
class motor_reset (threading.Thread):
    def run(self):
        global motor_rstflag
        msg1='RSTART'
        msg2='REND'
        client.send(msg1.encode('utf-8'))
        motor0.location_rst()
        motor1.location_rst()
        motor_rstflag=0
        motor0_stopflag=0
        motor1_stopflag=0
        #time.sleep(5)
        while True:
            input=ioin.read(0x00)
            if input&0x01==1:
                motor0_stopflag=1
                motor0.stop()
            if input&0x02==2:
                motor1_stopflag=1
                motor1.stop()
            if(motor0_stopflag==1&motor1_stopflag==1):
                motor_rstflag=1
                motor0_stopflag=0
                motor1_stopflag=0
                client.send(msg2.encode('utf-8'))
                break
                
class motor_move(threading.Thread):
    def __init__(self,dirx,diry,x,y):
        threading.Thread.__init__(self)
        self.dirx=dirx
        self.diry=diry
        self.x=x
        self.y=y
             
    def run(self):  
        msg1='MSTART'
        msg2='MEND'
        motor0_moveflag=0
        motor1_moveflag=0
        client.send(msg1.encode('utf-8'))
        motor0.motor_run(self.dirx,5000,0)
        motor1.motor_run(self.diry,5000,0)
        while True:
            if (self.x*1280)<=motor0.step():
                motor0_moveflag=1
                motor0.stop()
                
            if (self.y*1280)<=motor1.step():
                motor1_moveflag=1
                motor1.stop()
                
            if(motor0_moveflag==1&motor1_moveflag==1):
                motor0_moveflag=0
                motor1_moveflag=0
                break               

        client.send(msg2.encode('utf-8'))

        
class motor_scan(threading.Thread):
    def __init__(self,xmax,ymax):
        threading.Thread.__init__(self)
        self.xmax=xmax
        self.ymax=ymax
         
    def run(self):
        global motor_scanflag
        global scan_cnt,dir_scan
        scan_cnt=0
        dir_scan=1
        x_cnt=math.ceil(self.xmax/20)
        motor_scanflag=0
        for scan_cnt in range(0,x_cnt):
            motor1.motor_run(dir_scan,5000,self.ymax)
            if dir_scan==0:
                dir_scan=1
            else:
                dir_scan=0
            if scan_cnt<(x_cnt-1):
                motor0.motor_run(1,5000,20)
            else:
                pass
        motor_scanflag=1
            
            
        
class tcp(threading.Thread):
    def run(self):
        while True:
            global xpos_last,ypos_last
            msg_reset='RESET'
            msg_move='MOVE'
            msg_loc='LOCATION'
            msg_scan='SCAN'
#             msg_test='abc'
            msg = client.recv(1024)
            msg=msg.decode('utf-8')
            dir_x=0
            dir_y=0
            
            if msg.find(msg_reset)!=-1:         #复位
                thread1=motor_reset()
                thread1.start()
                  
            elif msg.find(msg_move)!=-1:        #移动
                xflag=0
                yflag=0
                xlist=[]
                ylist=[]
                for i in msg:
                    if xflag==1:
                        xlist.append(i)
                    if yflag==1:
                        ylist.append(i)
                    if i=='(':
                        xflag=1
                    if i==',':
                        xflag=0
                        yflag=1
                    if i==')':
                        yflag=0
                xlist=xlist[0:-1]
                ylist=ylist[0:-1]
                xpos=float(''.join(xlist))#浮点型x坐标
                ypos=float(''.join(ylist))#浮点型y坐标
                if(xpos-xpos_last<=0):
                    dir_x=0
                else:
                    dir_x=1
                    
                if(ypos-ypos_last<=0):
                    dir_y=0
                else:
                    dir_y=1
                
                if(motor_rstflag==0):
                    thread1=motor_reset()
                    thread1.start()
                    while True:
                        if(motor_rstflag==1):
                            break
                thread2=motor_move(dir_x,dir_y,abs(xpos-xpos_last),abs(ypos-ypos_last))
                xpos_last=xpos
                ypos_last=ypos
                thread2.start()
                
                
            elif msg.find(msg_scan)!=-1:          #扫描
                xflag=0
                yflag=0
                xlist=[]
                ylist=[]
                for i in msg:
                    if xflag==1:
                        xlist.append(i)
                    if yflag==1:
                        ylist.append(i)
                    if i=='(':
                        xflag=1
                    if i==',':
                        xflag=0
                        yflag=1
                    if i==')':
                        yflag=0
                xlist=xlist[0:-1]
                ylist=ylist[0:-1]
                x_max=int(''.join(xlist))#扫描长度
                y_max=int(''.join(ylist))#扫描宽度
                thread1=motor_reset()
                thread1.start()
                time.sleep(0.5)
                while True:
                    if(motor_rstflag==1):
                        break
                thread3=motor_scan(x_max,y_max)
                thread3.start()
#                 while True:
#                     if(motor_scanflag==1):
#                         break
#                 thread1=motor_reset()
#                 thread1.start()
                
            elif msg.find(msg_loc)!=-1:            #发送坐标
                x_pos=float(scan_cnt*20+motor0.step()/1280)
                if dir_scan==1:
                    y_pos=float(motor1.step()/1280)
                else:
                    y_pos=float(y_max-motor1.step()/1280)
                st='LOCATION'
                le='('
                di=','
                ri=')'
                lo=st+le+str(x_pos)+di+str(y_pos)+ri
                client.send(lo.encode('utf-8'))
                
#             elif msg.find(msg_test)!=-1:
#                 print("ok")

In [23]:
thread5 = tcp()

In [24]:
thread5.start()

In [11]:
stop_thread(thread5)

In [ ]:
def fun():
    async def motor_run(): 
        motor1.clear(1)
        motor1.clear(0)
        motor1.run(5000000)
        motor1.en(1)
    async def check():
            input=ioin.read(0x00)
            if input==15:
                print("motor stop!")
                motor1.en(0)
    loop =  asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    tasks=asyncio.gather(motor_run(),check())
    loop.run_until_complete(tasks)   

In [79]:
motor1.clear(1)

In [86]:
motor0.dire(1)

In [45]:
motor1.power_en(1)

In [46]:
motor1.pwm_en(0)

In [42]:
motor1.motor_run(1,20000,0)

In [11]:
motor0.motor_run(1,20000,0)

In [44]:
motor1.step()

4951

In [10]:
motor1.stop()